In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from snmfem.estimator.nmf import NMF
from snmfem.measures import KLdiv
from snmfem.toy import create_toy_problem

In [ ]:
l = 25
k = 3
p = 100
c = 10
n_poisson=2000
G, P, A, X, Xdot = create_toy_problem(l, k, p, c, n_poisson)
G.shape
# G = np.eye(c)
# A = np.eye(k)
# X = G @ P @ A

In [ ]:
from pyunlocbox import functions
from pyunlocbox.solvers import mlfbf, solve
from snmfem.conf import log_shift
# Let us solve the following problem
# 
# \argmin_P  - \sum_ij  X_ij log(GPA)_ij + (GPA)_ij  s.t. P \geq 0

# \argmin_A  - \sum_ij  X_ij log(GPA)_ij + (GPA)_ij + f(A) s.t. A \geq 0


# class smooth(functions.func):
#     def __init__(self, G, A):
#         self.A = A
#         self.G = G
#         self.nabla = np.sum(self.G.T, axis=1, keepdims=True) @ np.sum(self.A.T, axis=0, keepdims=True)

#     def _eval(self, P):
#         return np.sum(self.G @ P @ self.A)
#     def _grad(self, P):
#         return self.nabla

# f_smooth = smooth(G, A)
# assert(f_smooth.grad(P).shape == P.shape)
# np.testing.assert_allclose(f_smooth.grad(P),f_smooth.grad(2*P))


class mxlogvpv(functions.func):
    def __init__(self, X, lambda_=1, log_shift=log_shift, **kwargs):
        super(mxlogvpv, self).__init__(**kwargs)
        self.lambda_ = lambda_
        self.X = X
        self.log_shift = log_shift
        self.offset = np.sum(self.X * np.log(self.X+log_shift)) - np.sum(self.X)

    def _eval(self, P):
        return self.offset * (self.offset - self.lambda_ * np.sum(self.X * np.log(P+log_shift)) + np.sum(P))
    def _prox(self, P, T):
        gamma = self.lambda_ * T
        delta = (P - gamma)**2 + 4 * gamma * self.X
        return (P-gamma + np.sqrt(delta))/2

def solve_P(G, A, X, **kwargs):
    L = lambda P : G @ P @ A
    Lt = lambda X: G.T @ X @ A.T 

    f = functions.proj_positive()
    g = mxlogvpv(X)
    h = functions.dummy()
    beta = 0
    mu = beta + np.linalg.norm(G,2) * np.linalg.norm(A,2)
    step = 1 / mu / 2

    x0 = np.zeros(P.shape)
    D = np.linalg.lstsq(A.T, X.T)[0].T
    x0 = np.abs(np.linalg.lstsq(G, D)[0])
    d0 = np.zeros(X.shape)

    solver = mlfbf(step=step, L=L, Lt=Lt, d0=d0 )
    ret = solve([f, g, h], x0, solver, **kwargs)

    sol = ret["sol"], np.array(ret["objective"]).sum(axis=1)
    return sol

In [ ]:
sol,  objective = solve_P(G, A, X, maxit=1000, rtol=1e-15)

plt.plot(objective)
plt.yscale("log")

np.linalg.norm(sol - P, "fro") / np.linalg.norm(P, "fro"), KLdiv(X, G @ sol, A)

In [ ]:
sol,  objective = solve_P(G, A, Xdot, maxit=2, rtol=1e-15)

plt.plot(objective)
# plt.yscale("log")

np.linalg.norm(sol - P, "fro") / np.linalg.norm(P, "fro"), KLdiv(X, G @ sol, A)

In [ ]:
sol

In [ ]:
from pyunlocbox import functions


In [ ]:
est = NMF(G=G, n_components=k, debug=True, max_iter=200,  force_simplex=True)

In [ ]:
Ps, As = est.fit_transform(X, eval_print=20)

In [ ]:
# est = NMF(G=G, n_components=k, debug=True, max_iter=2000, force_simplex=False)
# Pss, Ass = est.fit_transform(Xdot, eval_print=200)
# Ainit = Ass/np.sum(Ass, axis=0, keepdims=True)
        # - 'random': non-negative random matrices, scaled with:
        #     sqrt(X.mean() / n_components)
        # - 'nndsvd': Nonnegative Double Singular Value Decomposition (NNDSVD)
        #     initialization (better for sparseness)
        # - 'nndsvda': NNDSVD with zeros filled with the average of X
        #     (better when sparsity is not desired)
        # - 'nndsvdar': NNDSVD with zeros filled with small random values
        #     (generally faster, less accurate alternative to NNDSVDa
        #     for when sparsity is not desired)
est = NMF(G=G, n_components=k, debug=True, mu=0, max_iter=2000,tol=0, force_simplex=False, init="nndsvd")
Pss, Ass = est.fit_transform(Xdot, eval_print=200)
# np.sum(Ass, axis=0)

In [ ]:
# est = NMF(G=G, n_components=k, debug=True, max_iter=10, force_simplex=True)
# Pss, Ass = est.fit_transform(Xdot, A=A, eval_print=1)

In [ ]:
vmin = 0
vmax = 1

plt.subplot(2,1,1)
plt.imshow(A, vmin=vmin, vmax=vmax)
plt.colorbar()
plt.subplot(2,1,2)
plt.imshow(Ass, vmin=vmin, vmax=vmax)
plt.colorbar()
plt.show()

In [ ]:
np.min(Ass)

In [ ]:
k = 15
num = np.random.rand(k,23)/20
denum = np.random.rand(k,23)

def f(nu):
    sol = 0
    for n,d in zip(num, denum):
        sol = sol + n/(d+nu)
    return sol - 1

In [ ]:
r = np.sum(num/denum, axis=0)
# t = np.sum(num)
# if r<1:
#     bmax = 0
# else:
ind_min = np.argmax(num/denum, axis=0)
ind_min2 = np.argmin(denum, axis=0)
ind = np.arange(len(ind_min))

bmin1 = num[ind_min, ind]-denum[ind_min, ind]
bmin2 = num[ind_min2, ind]-denum[ind_min2, ind]
bmin = np.maximum(bmin1, bmin2)
bmax = r

# nu = np.arange(bmin, bmax, 1e-3)
# plt.plot(nu, f(nu))
# plt.show()

In [ ]:
bmin, bmax

In [ ]:
bmin

In [ ]:
ind_max = np.argmin(num/denum)
num[ind_max]/denum[ind_max]